In [2]:
import glob
import os
import pickle
import datetime

import numpy as np
import pandas as pd

from mymodule import base_feature, except_outlier,\
                     pre_train_test, load_model, ensemble, pre_submit

## preparation for submit

In [3]:
def preprocess(df,
               train=True,
               drops=[],
               save=False,
               save_name=None):
    # featuring from mymodule.py
    df = base_feature(df)  # add featuring processes

    #split data for training
    idx, x, y = pre_train_test(df, train=train,  drops=drops)

    # saving preprocessed_data
    if save:
        if save_name is None: raise NameError ('(arg: save_name) is None')
        df.to_csv('dataset/featured/' + save_name, index=False)
    return  idx, x, y

In [4]:
def main(data, drops, model_path, submit_path):
    idx, x, y = preprocess(data, drops=drops, train=False,)
    print('-'*20, 'preprocess done', '-'*20)
    
    models = load_model(model_path)
    print('-'*20, 'model are loaded', '-'*20)
    
    tm = ensemble(models,x)
    print('-'*20, 'out put is ensembled', '-'*20)

    pre_submit(idx, tm, submit_path)
    print('-'*20, 'all is done', '-'*20)

In [5]:
now = datetime.datetime.now()
now = now.strftime('%m_%d_%H_%M')
name = 'cluster_test'

data = pd.read_csv('dataset/original/test.csv')
drops = ['protein_sequence', 'pH', 'B', 'J', 'O', 'U', 'X', 'Z']
model_path = 'models/test/*'
submit_path = f'dataset/submit/{name}{now}.csv'

main(data, drops, model_path, submit_path)

/Users/lino/Desktop/kaggle/Novozymes_cmp/training/mymodule/mymodule.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[amino].loc[idx] = trg.count(amino)


-------------------- preprocess done --------------------
-------------------- model are loaded --------------------
-------------------- out put is ensembled --------------------
-------------------- all is done --------------------


In [6]:
submit = pd.read_csv(submit_path)
submit

,seq_id,tm
0,31551,46.7
1,32225,46.7
2,31762,46.7
3,32412,46.7
4,33252,46.8
...,...,...
2408,33327,49.5
2409,32975,49.5
2410,32560,49.5
2411,33270,49.5


In [10]:
submit['tm'] = range(len(submit))
submit.to_csv('dataset/submit/rank.csv', index=False)